## Koble data med join-funksjonen i pyspark
<font size=2>Når du skal koble sammen to datasett ved hjelp av koblingsnøkkel eller identifiseringsvariabler slik at datasettet består av variablene fra begge datasett, kan du bruke pyspark funksjonen:
<code>join</code>: <i>Joins two SparkDataFrames based on the given join expression.</i> ([spark.apache.org/join](https://spark.apache.org/docs/latest/api/R/join.html))

Syntaks:
<code><i>dataframe A</i>.join(<i>dataframe B</i>, '<i>kolonnennavn (enhetsnøkkelen)</i>', 'inner|outer|left|right')</code>

Eksemplene under kobler sammen datasettene <code>areal</code> og <code>innbyggerantall</code>. Enhetene og nøkkelen er <i>Land</i>. 
- Det første eksempelet kobler land som er med i begge settene. Det nye datasettet inneholder snittet av land (koblingsmetoden 'inner').
- Det andre eksempelet kobler alle landene sammen. Det nye datasettet inneholder unionen av land (koblingsmetode = 'outer').
</font>

#### Importere biblioteker (kode)

In [ ]:
from pyspark.sql.types import *
from pyspark import Row
from pyspark.sql import DataFrame

import pyspark.sql.functions as F

#### Hente eksempeldata (kode)

In [ ]:
df_areal = spark.read.path("/felles/veiledning/pyspark/eksempler/areal")
df_bnp = spark.read.path("/felles/veiledning/pyspark/eksempler/bnp")
df_innbyggerantall = spark.read.path("/felles/veiledning/pyspark/eksempler/innbyggerantall/2020")

### Kontrollere om det er konflikter i variabelnavnene (kolonnene)
<font size=2>Når to datasett skal kobles, er det viktig at kolonnene, som ikke er koblingsnøkkel har forskjellige navn. Dette kan løses på forskjellige måter. I dette eksemplet sjekker vi først hva kolonnene heter og deretter endrere vi kolonnennavnene for å unngå konflikt. For å sjekke kolonnenavn kan vi bruke pyspark-funksjonen for å skrive ut <i>skjemaet</i> til datasettet: <code><i>dataframe</i>.printSchema()</code>
</font>

#### Viser skjemaet til df_areal og df_innbyggerantall (kode)

In [ ]:
df_areal.printSchema()

df_innbyggerantall.printSchema()

### Endre kolonnennavn (variabelnavn)
<font size=2>I eksempelet med datasettene <code>df_areal</code> og <code>df_innbyggerantall</code> ser vi at følgende variabler (kolonnenavn) som bør endres:
- <code>df_areal.Årstall</code> er ikke i konflikt med <code>df_innbyggerantall.År</code>, men kan gjerne endres til forståelige navn, som f.eks. <code>df_areal.Registrert_areal</code> og <code>df_innbyggerantall.Registrert_innbyggerantall</code>
- <code>df_areal.Kilde</code> --> <code>df_areal.Kilde_areal</code>
- <code>df_innbyggerantall.Kilde</code> --> <code>df_innbyggerantall.Kilde_innbyggerantall</code>

Land og landkode er konblingsnøkler, så de kan være like.

For å endre navn på en kolonne (variabel) i datasettet, brukes pyspark-funksjonen <code><i>dataframe</i>.withColumnRenamed('<i>kolonnenavn</i>','<i>nytt kolonnenavn</i>')</code></font>

#### Endre kolonnenavn med pyspark-funksjonen withColumnRenamed (kode)

In [ ]:
df_areal = df_areal.withColumnRenamed('Årstall','Registrert_areal')
df_areal = df_areal.withColumnRenamed('Kilde','Kilde_areal')
df_innbyggerantall= df_innbyggerantall.withColumnRenamed('År', 'Registrert_innbyggerantall')
df_innbyggerantall= df_innbyggerantall.withColumnRenamed('Kilde', 'Kilde_innbyggerantall')

df_areal.printSchema()
df_innbyggerantall.printSchema()

#### Koble land som har areal og innbyggertall (snitt = 'inner') (kode)

In [ ]:
df_areal_innbyggerantall = df_areal.join(df_innbyggerantall, 'Land', 'inner')
df_areal_innbyggerantall.show()

#### Koble alle landene sammen (union = 'outer') (kode)

In [ ]:
df_areal_innbyggerantall_union = df_areal.join(df_innbyggerantall, 'Land','outer')
df_areal_innbyggerantall_union.show()